# Код для создания набора данных

In [ ]:
import json
import pandas as pd
import numpy as np
from PIL import Image, ImageDraw

In [ ]:
# Преобразование файла JSON в датасет с RGB
with open('_annotations.coco.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

image_data = data['images'][0]
image_id = image_data['id']
image_height = image_data['height']
image_width = image_data['width']
file_name = image_data['file_name']
image = Image.open(file_name).convert('RGB') 
image_array = np.array(image) 
pixel_classes = np.zeros((image_height, image_width), dtype=np.uint8)
for ann in data['annotations']:
    if ann['image_id'] == image_id:
        segmentation = ann['segmentation']
        if segmentation:
            mask = Image.new('L', (image_width, image_height), 0)
            draw = ImageDraw.Draw(mask)
            for seg in segmentation:
                draw.polygon(seg, outline=1, fill=1)
            pixel_classes += np.array(mask)

pixel_classes = np.clip(pixel_classes, 0, 1)
rows = []
for y in range(image_height):
    for x in range(image_width):
        rows.append({
            "x": x,
            "y": y,
            "R": image_array[y, x, 0],
            "G": image_array[y, x, 1],
            "B": image_array[y, x, 2],
            "class": pixel_classes[y, x]
        })

df = pd.DataFrame(rows)
df['brightness'] = 0.299 * df['R'] + 0.587 * df['G'] + 0.114 * df['B']
columns = [col for col in df.columns if col != 'class'] + ['class']
df = df[columns]
df.to_csv('road_pixels_with_brightness.csv', index=False)